In [19]:
!pip install easyocr
!pip install imutils
!pip install opencv-python
!pip install matplotlib

# fast-plate-ocr model
!pip install fast_plate_ocr

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.8/16.8 MB 17.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 9.7 MB/s eta 0:00:00


In [9]:
# camera feed
import matplotlib.pyplot as plt
import cv2
import numpy as np
from IPython.display import display, Image
import ipywidgets as widgets
import threading

cap = cv2.VideoCapture(0)
cap.set(3,640) # adjust width
cap.set(4,480) # adjust height

while True:
    success, img = cap.read()
    cv2.imshow("Webcam", img) # This will open an independent window
    if cv2.waitKey(1) & 0xFF==ord('q'): # quit when 'q' is pressed
        cap.release()
        break
        
cv2.destroyAllWindows() 
cv2.waitKey(1)

-1

In [ ]:
# licence recognition
import cv2 
from matplotlib import pyplot as plt
import numpy as np
import easyocr
import imutils
import random

img = cv2.imread("Test1.img") #read image
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
plt.title('Original Image')
plt.show()

gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) #convert image to gray
bfilter = cv2.bilateralFilter(gray, 11, 17, 17) #Noise reduction
plt.imshow(cv2.cvtColor(bfilter, cv2.COLOR_BGR2RGB)) #show processed image
plt.title('Processed Image')


edged = cv2.Canny(bfilter, 30, 200) #Edge detection
plt.imshow(cv2.cvtColor(edged, cv2.COLOR_BGR2RGB))

keypoints = cv2.findContours(edged.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE) #Find contours 
contours = imutils.grab_contours(keypoints) #Grab contours 
contours = sorted(contours, key=cv2.contourArea, reverse=True)[:10] #Sort contours

#Loop over our contours to find the best possible approximate contour of 10 contours
location = None
for contour in contours:
    approx = cv2.approxPolyDP(contour, 10, True)
    if len(approx) == 4:
        location = approx
        break
     
print("Location: ", location)

mask = np.zeros(gray.shape, np.uint8) #create blank image with same dimensions as the original image
new_image = cv2.drawContours(mask, [location], 0,255, -1) #Draw contours on the mask image
new_image = cv2.bitwise_and(img, img, mask=mask) #Take bitwise AND between the original image and mask image

plt.imshow(cv2.cvtColor(new_image, cv2.COLOR_BGR2RGB)) #show the final image

(x,y) = np.where(mask==255) #Find the co-ordinates of the four corners of the document
(x1, y1) = (np.min(x), np.min(y)) #Find the top left corner
(x2, y2) = (np.max(x), np.max(y)) #Find the bottom right corner
cropped_image = gray[x1:x2+1, y1:y2+1] #Crop the image using the co-ordinates

plt.imshow(cv2.cvtColor(cropped_image, cv2.COLOR_BGR2RGB)) #show the cropped image

reader = easyocr.Reader(['en']) #create an easyocr reader object with english as the language
result = reader.readtext(cropped_image) #read text from the cropped image

text = result[0][-2] #Extract the text from the result
font = cv2.FONT_HERSHEY_SIMPLEX #Font style
res = cv2.putText(img, text=text, org=(approx[0][0][0], approx[1][0][1]+60), fontFace=font, fontScale=1, color=(0,255,0), thickness=2, lineType=cv2.LINE_AA) #put the text on the image
res = cv2.rectangle(img, tuple(approx[0][0]), tuple(approx[2][0]), (0,255,0),3) #Draw a rectangle around the text

plt.imshow(cv2.cvtColor(res, cv2.COLOR_BGR2RGB)) #show the final image with text

In [17]:
# gpt ver

import cv2 
import numpy as np
import easyocr
import imutils
import matplotlib.pyplot as plt

# Initialize the EasyOCR reader
reader = easyocr.Reader(['en'])

# Function to process each frame
def process_frame(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)  # Convert image to gray
    bfilter = cv2.bilateralFilter(gray, 11, 17, 17)  # Noise reduction
    edged = cv2.Canny(bfilter, 30, 200)  # Edge detection

    keypoints = cv2.findContours(edged.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)  # Find contours 
    contours = imutils.grab_contours(keypoints)  # Grab contours 
    contours = sorted(contours, key=cv2.contourArea, reverse=True)[:10]  # Sort contours

    # Loop over contours to find the best possible approximate contour of a license plate
    location = None
    for contour in contours:
        approx = cv2.approxPolyDP(contour, 10, True)
        if len(approx) == 4:
            location = approx
            break

    if location is not None:
        mask = np.zeros(gray.shape, np.uint8)  # Create a blank image with the same dimensions as the original image
        new_image = cv2.drawContours(mask, [location], 0, 255, -1)  # Draw contours on the mask image
        new_image = cv2.bitwise_and(img, img, mask=mask)  # Take bitwise AND between the original image and mask image

        (x, y) = np.where(mask == 255)  # Find the coordinates of the four corners of the document
        (x1, y1) = (np.min(x), np.min(y))  # Find the top left corner
        (x2, y2) = (np.max(x), np.max(y))  # Find the bottom right corner
        cropped_image = gray[x1:x2 + 1, y1:y2 + 1]  # Crop the image using the coordinates

        result = reader.readtext(cropped_image)  # Read text from the cropped image
        if result:
            text = result[0][-2]  # Extract the text from the result
            font = cv2.FONT_HERSHEY_SIMPLEX  # Font style
            img = cv2.putText(img, text, (approx[0][0][0], approx[1][0][1] + 60), font, 1, (0, 255, 0), 2, cv2.LINE_AA)  # Put the text on the image
            img = cv2.rectangle(img, tuple(approx[0][0]), tuple(approx[2][0]), (0, 255, 0), 3)  # Draw a rectangle around the text
    return img

# Capture video from webcam
cap = cv2.VideoCapture(0)
cap.set(3, 640)  # Adjust width
cap.set(4, 480)  # Adjust height

while True:
    success, img = cap.read()
    if not success:
        break

    img = process_frame(img)  # Process the frame for license plate recognition

    cv2.imshow("Webcam - License Plate Recognition", img)  # Show the processed frame
    if cv2.waitKey(1) & 0xFF == ord('q'):  # Quit when 'q' is pressed
        break

cap.release()
cv2.destroyAllWindows()


In [7]:
import cv2
import numpy as np
import easyocr
import imutils
import matplotlib.pyplot as plt
from fast_plate_ocr import ONNXPlateRecognizer

# Initialize the EasyOCR reader
reader = easyocr.Reader(['en'])

# Initialize the ONNXPlateRecognizer outside the function for efficiency
recognizer = ONNXPlateRecognizer('argentinian-plates-cnn-model')

# Function to process each frame
def process_frame(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)  # Convert image to gray
    bfilter = cv2.bilateralFilter(gray, 11, 17, 17)  # Noise reduction
    edged = cv2.Canny(bfilter, 30, 200)  # Edge detection

    keypoints = cv2.findContours(edged.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)  # Find contours 
    contours = imutils.grab_contours(keypoints)  # Grab contours 
    contours = sorted(contours, key=cv2.contourArea, reverse=True)[:10]  # Sort contours

    # Loop over contours to find the best possible approximate contour of a license plate
    location = None
    for contour in contours:
        approx = cv2.approxPolyDP(contour, 10, True)
        if len(approx) == 4:
            location = approx
            break

    if location is not None:
        mask = np.zeros(gray.shape, np.uint8)  # Create a blank image with the same dimensions as the original image
        cv2.drawContours(mask, [location], 0, 255, -1)  # Draw contours on the mask image
        new_image = cv2.bitwise_and(img, img, mask=mask)  # Take bitwise AND between the original image and mask image

        (x, y) = np.where(mask == 255)  # Find the coordinates of the four corners of the document
        (x1, y1) = (np.min(x), np.min(y))  # Find the top left corner
        (x2, y2) = (np.max(x), np.max(y))  # Find the bottom right corner
        cropped_image = gray[x1:x2 + 1, y1:y2 + 1]  # Crop the image using the coordinates

        # Use fast-plate-ocr 
        text = recognizer.run(cropped_image)
        if text:
            text = text[0]
        else:
            text = "No text detected"

        # Draw the detected text on the original image
        cv2.putText(new_image, text, (y1, x1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

        # Draw the mask directly on the original image
        cv2.drawContours(img, [location], -1, (0, 255, 0), 2)  # Draw contour in green on original image

        return img  # Return the modified image with the mask drawn
    return img  # If no location is found, return the original image

# Capture video from webcam
cap = cv2.VideoCapture(0)
cap.set(3, 640)  # Adjust width
cap.set(4, 480)  # Adjust height

while True:
    success, img = cap.read()
    if not success:
        break

    img = process_frame(img)  # Process the frame for license plate recognition

    cv2.imshow("Webcam - License Plate Recognition", img)  # Show the processed frame
    if cv2.waitKey(1) & 0xFF == ord('q'):  # Quit when 'q' is pressed
        break

cap.release()
cv2.destroyAllWindows()


2024-09-26 13:25:07.335832 [W:onnxruntime:, coreml_execution_provider.cc:107 GetCapability] CoreMLExecutionProvider::GetCapability, number of partitions supported by CoreML: 2 number of nodes in the graph: 49 number of nodes supported by CoreML: 39
